In [97]:
import os
import json
#from pandas import json_normalize
import pandas as pd
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions
from dotenv import load_dotenv
load_dotenv()

ModuleNotFoundError: No module named 'ibm_watson'

In [ ]:
file = pd.read_csv("nyt_concatenated_data.csv")

In [ ]:
headline_emotion = []
lead_paragraph_emotion = []
headline_sentiment = []
lead_paragraph_sentiment = []

In [ ]:
# Get the Tone Analyzer API Key and URL
tone_api = os.getenv("tone_key")
tone_url = os.getenv("tone_url")

In [ ]:
authenticator = IAMAuthenticator(tone_api)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)

natural_language_understanding.set_service_url(tone_url)
for i, j in file.iterrows():
    response = natural_language_understanding.analyze(
        text= j.Headline,
        features=Features(emotion =EmotionOptions(), sentiment=SentimentOptions())).get_result()
#print(json.dumps(response, indent=2))
    headline_emotion.append(response['emotion']['document']['emotion'])
    headline_sentiment.append(response['sentiment']['document'])

In [ ]:
natural_language_understanding.set_service_url(tone_url)
for i, j in file.iterrows():
    response = natural_language_understanding.analyze(
        text= j['Lead Paragraph'],
        features=Features(emotion =EmotionOptions(), sentiment=SentimentOptions())).get_result()
#print(json.dumps(response, indent=2))
    lead_paragraph_emotion.append(response['emotion']['document']['emotion'])
    lead_paragraph_sentiment.append(response['sentiment']['document'])

In [9]:
df_lead_paragraph_sent = pd.DataFrame(lead_paragraph_sentiment)
df_lead_paragraph_emotion = pd.DataFrame(lead_paragraph_emotion)

In [20]:
df_lead_paragraph_sent.rename(columns={"label": "emotion_lead_par", "score": "score_lead_par"}, inplace = True)
df_lead_paragraph_emotion.rename(columns = {"anger":"anger_lead_par","disgust":"disgust_lead_par", "fear" :"fear_lead_par", "joy": "joy_lead_par", "sadness": "sadness_lead_par"}, inplace = True)

In [24]:
df_headline_sent = pd.DataFrame(headline_sentiment)
df_headline_emotion = pd.DataFrame(headline_emotion)

In [25]:
df_headline_sent.rename(columns={"label": "emotion_headline", "score": "score_headline"}, inplace = True)
df_headline_emotion.rename(columns = {"anger":"anger_headline","disgust":"disgust_headline", "fear" :"fear_headline", "joy": "joy_headline", "sadness": "sadness_headline"}, inplace = True)


In [30]:
df_concat1 = pd.concat([df_headline_sent,df_headline_emotion], join = 'inner', axis = 1)
df_concat2 = pd.concat([df_lead_paragraph_sent, df_lead_paragraph_emotion], join = 'inner', axis =1)
df_concat3 = pd.concat([df_concat1, df_concat2], join = 'inner', axis =1)
df_nlp = pd.concat([df_concat3, file], join = 'inner', axis = 1)
df_nlp.set_index('Date', inplace = True)

In [32]:
df_nlp.to_csv(r'df_nlp.csv', index = True)

In [9]:
file2 = pd.read_csv("global_indicies.csv", infer_datetime_format = True, parse_dates = True )
file2["Date"]=pd.to_datetime(file2["Date"])
file2["Date"]=file2["Date"].dt.date
file2.set_index("Date",inplace=True)
file2.head()

,Ticker,Close
Date,,
2020-10-15,FTSE,5832.52
2020-10-16,FTSE,5919.58
2020-10-19,FTSE,5884.65
2020-10-20,FTSE,5889.22
2020-10-21,FTSE,5776.50


In [10]:
file3 = file2.pivot(columns = 'Ticker')

In [11]:
df_nlp = pd.read_csv("df_nlp.csv", index_col = 'Date')

In [12]:
file3.index = pd.to_datetime(file3.index)
df_nlp.index = pd.to_datetime(df_nlp.index)

In [13]:
#concat
final_df_1 = pd.concat([file3,df_nlp], join = 'inner', axis = 1 )
final_df_1.dropna(inplace = True)

In [28]:
final_df_prep = final_df_1.copy().drop(columns = ['Headline', 'Lead Paragraph'] )
final_df_encoded = pd.get_dummies(final_df_prep)
final_df_encoded.drop(columns = 'emotion_headline_negative', inplace = True)

In [30]:
final_df_encoded.to_csv(r'final_encoded_data.csv', index = True)

In [54]:
final_df_encoded.head()

,"(Close, CAC)","(Close, FTSE)","(Close, MDAX)","(Close, SP500)","(Close, TOPIX)",score_headline,anger_headline,disgust_headline,fear_headline,joy_headline,sadness_headline,score_lead_par,anger_lead_par,disgust_lead_par,fear_lead_par,joy_lead_par,sadness_lead_par,emotion_lead_par_negative,emotion_lead_par_positive
Date,,,,,,,,,,,,,,,,,,,
2020-10-16,4935.86,5919.58,27765.20,3483.81,1617.69,-0.486293,0.232385,0.498824,0.062049,0.575430,0.456423,-0.458753,0.134218,0.182121,0.111655,0.540156,0.546577,1,0
2020-10-19,4929.27,5884.65,27815.35,3426.92,1637.98,-0.695622,0.564387,0.564141,0.075140,0.166083,0.487874,-0.406186,0.485618,0.446847,0.097647,0.526350,0.528911,1,0
2020-10-20,4929.28,5889.22,27692.89,3443.12,1625.74,-0.529668,0.598165,0.278106,0.109942,0.525510,0.206216,-0.337775,0.471010,0.189398,0.115233,0.549269,0.520048,1,0
2020-10-21,4853.95,5776.50,27165.35,3435.56,1637.60,-0.701941,0.614077,0.474541,0.080264,0.102332,0.551506,-0.461309,0.173107,0.484750,0.533805,0.528012,0.500254,1,0
2020-10-22,4851.38,5785.65,27189.17,3453.49,1619.79,-0.580007,0.489271,0.264428,0.092988,0.190490,0.132476,-0.624506,0.161651,0.149247,0.120754,0.501384,0.514247,1,0


In [95]:
df_topix = final_df_encoded.iloc[:,4:]
df_sp500 = final_df_encoded.iloc[:,3:].drop(columns = (final_df_encoded.iloc[:,4:5]))
df_mdax = final_df_encoded.iloc[:,2:].drop(columns = (final_df_encoded.iloc[:,3:5]))
df_ftse = final_df_encoded.iloc[:,1:].drop(columns = (final_df_encoded.iloc[:,2:6]))
df_cac = final_df_encoded.iloc[:,0:].drop(columns = (final_df_encoded.iloc[:,1:7]))